# Tutorial: Electrothermal YAML Validation Modes

**Audience:** users configuring thermal ports and parser validation in CI.

**You will learn:**

- strict vs non-strict behavior for missing thermal constants,
- diagnostics for unsupported thermal-capable components,
- how to inspect parser errors and warnings programmatically.


## Outline

1. Build a minimal helper to parse YAML strings.
2. Strict mode: missing `rth`/`cth` should fail.
3. Non-strict mode: defaults should be applied with warnings.
4. Unsupported component thermal enablement should fail deterministically.


In [ ]:
# Setup cell: keep it short and deterministic
try:
    import pulsim as ps
except Exception as exc:
    raise RuntimeError(
        "Failed to import pulsim. Run notebook with PYTHONPATH=build/python."
    ) from exc


def parse_string(yaml_text: str, strict: bool):
    opts = ps.YamlParserOptions()
    opts.strict = strict
    parser = ps.YamlParser(opts)
    circuit, sim_opts = parser.load_string(yaml_text)
    return parser, circuit, sim_opts


## Step 1 - Compare parser modes

The same incomplete thermal block behaves differently by parser mode:

- strict: error (`PULSIM_YAML_E_THERMAL_MISSING_REQUIRED`)
- non-strict: default values + warning (`PULSIM_YAML_W_THERMAL_DEFAULT_APPLIED`)


In [ ]:
# Minimal working example
missing_constants_yaml = """
schema: pulsim-v1
version: 1
simulation:
  tstop: 1e-4
  dt: 1e-6
  thermal:
    default_rth: 1.7
    default_cth: 0.3
components:
  - type: mosfet
    name: M1
    nodes: [gate, drain, source]
    thermal:
      enabled: true
"""

strict_parser, _, _ = parse_string(missing_constants_yaml, strict=True)
print("strict errors:")
for err in strict_parser.errors:
    print("-", err)
print("strict warnings:", len(strict_parser.warnings))

non_strict_parser, _, non_strict_opts = parse_string(missing_constants_yaml, strict=False)
print()
print("non-strict errors:", len(non_strict_parser.errors))
print("non-strict warnings:")
for warn in non_strict_parser.warnings:
    print("-", warn)

if "M1" in non_strict_opts.thermal_devices:
    cfg = non_strict_opts.thermal_devices["M1"]
    print()
    print("M1 defaults applied -> rth:", cfg.rth, "cth:", cfg.cth)

unsupported_yaml = """
schema: pulsim-v1
version: 1
simulation:
  tstop: 1e-4
  dt: 1e-6
components:
  - type: resistor
    name: R1
    nodes: [in, 0]
    value: 10
    thermal:
      enabled: true
      rth: 0.5
      cth: 0.1
"""

unsupported_parser, _, _ = parse_string(unsupported_yaml, strict=True)
print()
print("unsupported component errors:")
for err in unsupported_parser.errors:
    print("-", err)

assert any("PULSIM_YAML_E_THERMAL_MISSING_REQUIRED" in err for err in strict_parser.errors)
assert len(non_strict_parser.errors) == 0
assert any("PULSIM_YAML_W_THERMAL_DEFAULT_APPLIED" in warn for warn in non_strict_parser.warnings)
assert any("PULSIM_YAML_E_THERMAL_UNSUPPORTED_COMPONENT" in err for err in unsupported_parser.errors)


## Exercises

1. Add `rth` but keep `cth` missing in strict mode. Confirm the exact diagnostic path.
2. Set `rth: 0` and `cth: -1` and inspect range diagnostics.
3. Try a `bjt_npn` with thermal enabled and verify it is accepted.


In [ ]:
# Exercise answer scaffold
# Build your YAML strings and call parse_string(yaml_text, strict=True/False).
# Then print parser.errors / parser.warnings and inspect options.thermal_devices.

# TODO: implement your experiment here.
